In [78]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, List


#TODO: Fix your notation on what is len(knots) and the last knot. e.g if x = [x0, x1, ..., xn] then len(knots) = n+1/
#       Try except blocks for inputs

In [4]:
def neville_scheme(x:float, knots:np.ndarray, f:Callable)->float: # Cost O(n^2) | additional Mem O(n^2)
    """ With memorizing the whole matrix. Its better if you need to add knots later."""
    n = len(knots)
    p = np.zeros((n,n))
    p[:,0] = f(knots)

    for i in range(1,n):
        for j in range(n-i):
            p[i,j] = ((x-knots[j])*p[i-1, j+1]-(x-knots[i-1, j+i])*p[j])/(knots[j+i]-knots[j])
    return p[-1,0]    

In [2]:
def neville_scheme(x:float, knots:np.ndarray, f:Callable)->float: # Cost O(n^2) | add Mem O(n)
    n = len(knots)
    p = f(knots)

    for i in range(1,n):
        for j in range(n-i):
            p[j] = ((x-knots[j])*p[j+1]-(x-knots[j+i])*p[j])/(knots[j+i]-knots[j])
    return p[0]    

In [7]:
def Horner_scheme(x:float, knots:np.ndarray, coef: np.ndarray)->float: # Cost O(n) | add Mem O(1)
    n = len(knots)-1
    y = coef[n]
    for i in range(n-1,-1,-1):
        y = coef[i] +(x-knots[i])*y
    return y

In [8]:
def divided_differences(knots:np.ndarray, f: Callable, values:None|np.ndarray = None)->np.ndarray: # Cost O(n^2)
    if values == None:
        values = f(knots)
    n = len(knots)
    res = np.zeros((n,n))
    res[:,0] = values

    for i in range(1, n):
        for j in range(n-i):
            res[j,i] = (res[j,i-1]-res[j-1,i-1])/(knots[j]-knots[j-1])
    
    return res[0,:]    

In [9]:
def uniform_points(a:float, b:float, N:int)->np.ndarray:
    return np.linspace(a,b,N,endpoint=True)

In [29]:
def chebysev_points(a:float, b:float, N:int)->np.ndarray:
    x = np.pi * (2*np.arange(N)+1)/(2*N+2)
    return (b+a)/2 + (b-a)*np.cos(x)/2


In [37]:
def forward_substitution(L:np.ndarray, b:np.ndarray)->np.ndarray: # Cost O(n^2)
    x = np.zeros(len(b))
    for i in range(len(b)):
        x[i] = (b[i] - np.dot(L[i,:i-1], x[:i-1]))/L[i,i]
    return x

In [42]:
def backward_substitution(U:np.ndarray, b:np.ndarray)->np.ndarray: # Cost O(n^2)
    x = np.zeros(len(b))
    for i in range(len(b)-1,-1,-1):
        x[i] = (b[i] - np.dot(U[i,i-1:], x[i-1:]) ) / U[i,i]
    return x

In [88]:
def gaussian_elimination(A:np.ndarray)->List[np.ndarray]: # Cost O(n^3) | Mem here twice as much, because theoretically one can store everything on the initial matrix. No need for copying
    U = A.copy()
    L = np.eye(len(U))

    for i in range(len(U)):
        for j in range(i+1, len(U)):
            factor =  U[j,i]/U[i,i]
            U[j,i:] = U[j,i:]- factor*U[i,i:]
            L[j,i] = factor
            
    return [L, U]

In [139]:
def gaussian_elimination_with_pivoting(A:np.ndarray)->List[np.ndarray]:
    U = A.copy()
    n=len(U)
    L = np.eye(n)
    perm = list(range(n))

    for i in range(n):
        index_of_largest = np.argmax(np.abs(U[i:,i]))
        perm[i], perm[index_of_largest] = perm[index_of_largest], perm[i]

        for j in range(i+1, len(U)):
            factor =  U[perm[j],i]/U[perm[i],i]
            U[perm[j],i:] = U[perm[j],i:]- factor*U[perm[i],i:]
            L[perm[j],i] = factor
            
    return [L, U]

In [ ]:
list()

In [109]:
def crouts(A:np.ndarray)->List[np.ndarray]:
    n = len(A)
    U = np.zeros((n,n))
    L = np.eye(n)

    for i in range(n):
        for j in range(i,n):
            U[i,j] = A[i,j] - np.dot(L[i, :i-1], U[:i-1, j])
        
        for j in range(i+1, n):
            L[j, i] = (A[j,i]-np.dot(L[j, :i-1], U[:i-1, i]))/U[i,i]

    return [L,U]

In [117]:
def crouts_w_overwriting(A:np.ndarray):
    n = len(A)
    
    for i in range(n):
        for j in range(i,n):
            A[i,j] = A[i,j] - np.dot(A[i, :i-1], A[:i-1, j])
        
        for j in range(i+1, n):
            A[j, i] = (A[j,i]-np.dot(A[j, :i-1], A[:i-1, i]))/A[i,i]


In [125]:
def RandomSPD(n:int)->np.ndarray:
    A = np.random.random((n,n))
    A = np.matmul(A.T, A)
    return A+0.1*np.eye(n)

In [137]:
def cholesky(A:np.ndarray)->np.ndarray: # Needs to be fixed! # Half computations than LU as half terms need to be computed
    n = len(A)
    L = np.zeros((n,n))
    # L[0,0] = np.sqrt(A[0,0])
    # L = np.eye(n)

    for i in range(n):
        for j in range(i+1):
            sum_term = np.sum(L[i,:j]*L[j,:j])
            if i == j:
                L[i,j] = np.sqrt(A[i,i]-sum_term)
            else:
                L[i,j] = (A[i,i]-sum_term)/ L[j,j]
    return L

In [ ]:
def Gram_Schmidt():# You decide
    pass

In [ ]:
def Householder():
    pass

In [ ]:
def given_rotations():
    pass1

In [165]:
def gaussian_elimination_with_pivoting(A:np.ndarray)->List[np.ndarray]: # Problematic
    U = A.copy()
    n=len(U)
    L = np.eye(n)
    perm = list(range(n))

    for i in range(n-1):
        print(perm)
        print(L)
        print(U)
        print('-----------------')
        index_of_largest = np.argmax(np.abs(A[i:,i])) + i
        perm[i], perm[index_of_largest] = perm[index_of_largest], perm[i]

        for j in range(i+1, n):
            factor =  U[perm[j],i]/U[perm[i],i]
            L[perm[j],i] = factor
            for m in range(i+1, n):
                U[perm[j],m] = U[perm[j],m]- L[perm[j], i]*U[perm[i],m]
            
    return [L, U]

In [140]:
A = np.random.random((4,4))
A

array([[0.66430596, 0.74689251, 0.12184275, 0.36929198],
       [0.94085048, 0.6081773 , 0.64240597, 0.49317733],
       [0.42198853, 0.1753529 , 0.34194809, 0.57304141],
       [0.93723288, 0.1982729 , 0.60886183, 0.914059  ]])

In [141]:
L, U = gaussian_elimination(A)

In [166]:
l,u = gaussian_elimination_with_pivoting(A)

[0, 1, 2, 3]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[0.66430596 0.74689251 0.12184275 0.36929198]
 [0.94085048 0.6081773  0.64240597 0.49317733]
 [0.42198853 0.1753529  0.34194809 0.57304141]
 [0.93723288 0.1982729  0.60886183 0.914059  ]]
-----------------
[1, 0, 2, 3]
[[0.70606964 0.         0.         0.        ]
 [0.         1.         0.         0.        ]
 [0.44851817 0.         1.         0.        ]
 [0.99615497 0.         0.         1.        ]]
[[ 0.66430596  0.31747699 -0.3317406   0.02107445]
 [ 0.94085048  0.6081773   0.64240597  0.49317733]
 [ 0.42198853 -0.09742567  0.05381734  0.35184242]
 [ 0.93723288 -0.40756593 -0.03107407  0.42277795]]
-----------------
[1, 0, 2, 3]
[[ 0.70606964  0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.44851817 -0.30687473  1.          0.        ]
 [ 0.99615497 -1.28376528  0.          1.        ]]
[[ 0.66430596  0.31747699 -0.3317406   0.02107445]
 [ 0.94085048  0.6081773 

In [167]:
np.matmul(l,u)

array([[ 0.46904627,  0.22416086, -0.23423196,  0.01488003],
       [ 0.94085048,  0.6081773 ,  0.64240597,  0.49317733],
       [ 0.05354402, -0.05447094, -0.35096891, -0.10922505],
       [ 0.39115338, -0.87206655, -1.61211465, -0.16229792]])

In [159]:
l

array([[ 0.70606964,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.44851817, -0.30687473,  0.10501225,  0.        ],
       [ 0.99615497, -1.28376528,  0.        ,  1.        ]])

In [160]:
u

array([[ 0.        ,  0.31747699, -0.3317406 ,  0.02107445],
       [ 0.94085048,  0.6081773 ,  0.64240597,  0.49317733],
       [ 0.        ,  0.        ,  0.        ,  0.3110717 ],
       [ 0.        ,  0.        , -0.45695113,  0.44983259]])

In [120]:
np.matmul(L,U)

array([[0.6777794 , 0.88904444, 0.86691284, 0.39322293],
       [0.61429154, 0.95491742, 0.69707534, 0.76606492],
       [0.26043831, 0.84837119, 0.20780333, 0.28897602],
       [0.58874101, 0.73858399, 0.30016066, 0.76322206]])

In [121]:
crouts_w_overwriting(A)

In [122]:
A

array([[-0.55351531, -0.20328375,  1.30831908,  0.38853355],
       [-0.33633848,  0.95491742,  0.69707534,  0.76606492],
       [-1.14868518,  0.88842363,  1.71065006,  0.73527875],
       [-2.72408454,  0.77345326,  2.25886785,  1.22910488]])

In [124]:
U

array([[ 0.6777794 ,  0.88904444,  0.86691284,  0.39322293],
       [ 0.        ,  0.14915012, -0.08863343,  0.40967534],
       [ 0.        ,  0.        ,  0.1758318 , -1.2540375 ],
       [ 0.        ,  0.        ,  0.        , -2.85842739]])

In [123]:
L

array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.90632961,  1.        ,  0.        ,  0.        ],
       [ 0.38425233,  3.39760916,  1.        ,  0.        ],
       [ 0.8686322 , -0.2257366 , -2.68936302,  1.        ]])

In [43]:
U = np.array([
    [1, 2, 3, 4],
    [0, 1, 4, 5],
    [0, 0, 1, 6],
    [0, 0, 0, 1]
])

b = np.array([1,2,3,4])

In [44]:
backward_substitution(U,b)

array([-15.,  66., -21.,   4.])